In [1]:
import cifar10_data
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
def conv2d_maxpool(input_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
    """
    Apply convolution then max pooling to x_tensor
    :param input_tensor: TensorFlow Tensor
    :param conv_num_outputs: Number of outputs for the convolutional layer
    :param conv_ksize: kernal size 2-D Tuple for the convolutional layer
    :param conv_strides: Stride 2-D Tuple for convolution
    :param pool_ksize: kernal size 2-D Tuple for pool
    :param pool_strides: Stride 2-D Tuple for pool
    : return: A tensor that represents convolution and max pooling of x_tensor
    """
    conv_kernel = tf.Variable(tf.truncated_normal(shape=[*conv_ksize,input_tensor.get_shape().as_list()[-1], conv_num_outputs],stddev=0.1))
    conv = tf.nn.conv2d(input_tensor, conv_kernel, strides=[1, *conv_strides,1], padding='SAME')
    bias = tf.Variable(tf.zeros(conv_num_outputs))
    pool = tf.nn.max_pool(tf.nn.bias_add(conv, bias), [1,*pool_ksize,1], strides=[1,*pool_strides,1], padding='SAME')
    return pool

In [3]:
def flatten(input_tensor):
    """
    Flatten input_tensor to (Batch Size, Flattened Image Size)
    : input_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    batch_size, *img_size = input_tensor.get_shape().as_list()
    img_size = img_size[0] * img_size[1] * img_size[2]
    # Be careful, batch_size = None 
    return tf.reshape(input_tensor, [-1, img_size])

In [4]:
def fully_connected(input_tensor, num_outputs):
    """
    Apply a fully connected layer to input_tensor using weight and bias
    : input_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    weight = tf.Variable(tf.truncated_normal(shape=[input_tensor.get_shape().as_list()[-1], num_outputs], stddev=0.1))
    bias = tf.Variable(tf.zeros(num_outputs))
    return tf.add(tf.matmul(input_tensor, weight), bias)

In [5]:
input_ = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3], name='input_')
labels_ = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='labels_')
keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')
step = tf.Variable(0,dtype=tf.int32,name="global_step")

In [7]:
conv_pool1 = conv2d_maxpool(input_, 64, [3,3], [1,1], [2,2], [2,2])
relu1 = tf.nn.relu(conv_pool1)
conv_pool2 = conv2d_maxpool(relu1, 128, [3,3], [1,1], [2,2], [2,2])
relu2 = tf.nn.relu(conv_pool2)
flatten3 = flatten(relu2)
drop3 = tf.nn.dropout(flatten3, keep_prob=keep_prob)
fc3 = fully_connected(drop3, 1000)
relu3 = tf.nn.relu(fc3)
drop4 = tf.nn.dropout(relu3, keep_prob=keep_prob)
fc4 = fully_connected(drop4, 100)
relu4 = tf.nn.relu(fc4)
logits = fully_connected(relu4, 10)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_), name="cost")
tf.summary.scalar('cost',cost)

optimizer = tf.train.AdamOptimizer().minimize(cost, global_step=step)

prediction = tf.nn.softmax(logits, name="prediction")
compare = tf.equal(tf.argmax(prediction, axis=1), tf.argmax(labels_, axis=1))
accuracy = tf.reduce_mean(tf.cast(compare, tf.float32), name='accuracy')

summary=tf.summary.merge_all()

In [9]:
epochs = 10
n_batches = 5
batch_size = 200
prob = 0.5
iteration =0
saver = tf.train.Saver()
checkpoint_path = 'data/cifar_checkpoints/cifar.ckpt'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer=tf.summary.FileWriter('data/cifar_graph',sess.graph)
    for e in range(epochs):
        for i in range(1, n_batches + 1):
            for images,labels in cifar10_data.load_preprocess_batch(i, batch_size):
                iteration += 1
                loss, _, summaries, global_step = sess.run([cost, optimizer, summary, step], feed_dict={input_:images, labels_:labels, keep_prob:prob})
                writer.add_summary(summaries, global_step=global_step)
                print("Epoch {}, Batch {}, Iteration {}: cost {:.5f}".format(e+1, i, iteration, loss))
            batch = cifar10_data.load_preprocess_batch_data(i)
            valid_images = batch['valid_images']
            valid_labels = batch['valid_labels']
            val_acc = sess.run(accuracy, feed_dict={input_:valid_images, labels_:valid_labels, keep_prob:1.0})
            print("Epoch {}, Batch {}: val_acc {:.4f}".format(e+1, i, val_acc))
        saver.save(sess, checkpoint_path, global_step=step)
    writer.flush()
    writer.close()

Epoch 1, Batch 1, Iteration 1: cost 12.19203
Epoch 1, Batch 1, Iteration 2: cost 8.94740
Epoch 1, Batch 1, Iteration 3: cost 7.30936
Epoch 1, Batch 1, Iteration 4: cost 4.98976
Epoch 1, Batch 1, Iteration 5: cost 4.30131
Epoch 1, Batch 1, Iteration 6: cost 3.73847
Epoch 1, Batch 1, Iteration 7: cost 2.99436
Epoch 1, Batch 1, Iteration 8: cost 2.76711
Epoch 1, Batch 1, Iteration 9: cost 2.46167
Epoch 1, Batch 1, Iteration 10: cost 2.41995
Epoch 1, Batch 1, Iteration 11: cost 2.41241
Epoch 1, Batch 1, Iteration 12: cost 2.35278
Epoch 1, Batch 1, Iteration 13: cost 2.36409
Epoch 1, Batch 1, Iteration 14: cost 2.34314
Epoch 1, Batch 1, Iteration 15: cost 2.31425
Epoch 1, Batch 1, Iteration 16: cost 2.32712
Epoch 1, Batch 1, Iteration 17: cost 2.32704
Epoch 1, Batch 1, Iteration 18: cost 2.29519
Epoch 1, Batch 1, Iteration 19: cost 2.31190
Epoch 1, Batch 1, Iteration 20: cost 2.30381
Epoch 1, Batch 1, Iteration 21: cost 2.32453
Epoch 1, Batch 1, Iteration 22: cost 2.30173
Epoch 1, Batch 1, 